In [1]:
import os
import numpy as np
import gc
import pandas as pd
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from tensorflow import keras

In [3]:
df = pd.read_csv("../../../../Dataset/02-03-2018(Time).csv", low_memory = False)

In [4]:
df = df.drop([0,1])

In [5]:
df['Timestamp']= pd.to_datetime(df['Timestamp'])

In [6]:
df = df.sort_values(by=['Timestamp'])

In [7]:
df = df.drop(columns = ['Timestamp'])

In [8]:
input_label = np.array(df.loc[:, df.columns != "Label"]).astype(np.float)

In [9]:
output_label = np.array(df["Label"])

In [10]:
out = []
for o in output_label:
    if(o == "Benign"):out.append(0)
    else: out.append(1)
output_label = out

In [11]:
scaler = MinMaxScaler(feature_range=(0,1))
scaler.fit(input_label)
input_label = scaler.transform(input_label)

<h2>AutoEncoder</h2>

In [12]:
inp = shuffle(input_label)

In [13]:
inp_train,inp_test,out_train,out_test = train_test_split(inp, inp, test_size=0.2)

In [14]:
input_model = keras.layers.Input(shape = (78,))
enc = keras.layers.Dense(units = 64, activation = "relu", use_bias = True)(input_model)
enc = keras.layers.Dense(units = 36, activation = "relu", use_bias = True)(enc)
enc = keras.layers.Dense(units = 18, activation = "relu")(enc)
dec = keras.layers.Dense(units = 36, activation = "relu", use_bias = True)(enc)
dec = keras.layers.Dense(units = 64, activation = "relu", use_bias = True)(dec)
dec = keras.layers.Dense(units = 78, activation = "relu", use_bias = True)(dec)
auto_encoder = keras.Model(input_model, dec)

In [15]:
encoder = keras.Model(input_model, enc)
decoder_input = keras.layers.Input(shape = (18,))
decoder_layer = auto_encoder.layers[-3](decoder_input)
decoder_layer = auto_encoder.layers[-2](decoder_layer)
decoder_layer = auto_encoder.layers[-1](decoder_layer)
decoder = keras.Model(decoder_input, decoder_layer)

In [16]:
auto_encoder.compile(optimizer=keras.optimizers.Adam(lr=0.00025), loss = "mean_squared_error", metrics = ['accuracy'])

In [14]:
train = auto_encoder.fit(x = np.array(inp_train), y = np.array(out_train),validation_split= 0.1, epochs = 10, verbose = 2, shuffle = True)

Epoch 1/10
23502/23502 [==============================] - 30s 1ms/step - loss: 0.0159 - accuracy: 0.3499 - val_loss: 0.0145 - val_accuracy: 0.3834
Epoch 2/10
23502/23502 [==============================] - 26s 1ms/step - loss: 0.0145 - accuracy: 0.3671 - val_loss: 0.0145 - val_accuracy: 0.3553
Epoch 3/10
23502/23502 [==============================] - 24s 1ms/step - loss: 0.0145 - accuracy: 0.3708 - val_loss: 0.0145 - val_accuracy: 0.3774
Epoch 4/10
23502/23502 [==============================] - 24s 1ms/step - loss: 0.0145 - accuracy: 0.3707 - val_loss: 0.0145 - val_accuracy: 0.3605
Epoch 5/10
23502/23502 [==============================] - 24s 1ms/step - loss: 0.0145 - accuracy: 0.3718 - val_loss: 0.0145 - val_accuracy: 0.3684
Epoch 6/10
23502/23502 [==============================] - 24s 1ms/step - loss: 0.0145 - accuracy: 0.3720 - val_loss: 0.0145 - val_accuracy: 0.3904
Epoch 7/10
23502/23502 [==============================] - 24s 1ms/step - loss: 0.0145 - accuracy: 0.3734 - val_loss: 0

<h2>cross validation</h2>

In [17]:
input_label = encoder.predict(input_label)

In [18]:
inp = []
out = []
num = 0
for i in range(len(input_label) - 20 + 1):
    aux = []
    for j in range(i, i + 20):
        aux.append(input_label[j])
    inp.append(aux)
    out.append(output_label[i + 20 - 1])

In [19]:
input_label = np.array(inp)
output_label = np.array(out)

In [20]:
inp = None
out = None
gc.collect()

30

In [ ]:
def createModel():
    model = keras.Sequential([
        keras.layers.Input(shape = (20,18)),
        keras.layers.LSTM(units = 16, return_sequences = True, use_bias = True),
        keras.layers.LSTM(units = 8, return_sequences = False, use_bias = True),
        keras.layers.Dense(units = 2, activation = "softmax")
    ])
    model.compile(optimizer= keras.optimizers.Adam(lr= 0.00025), loss="sparse_categorical_crossentropy", metrics=['accuracy'])
    return model

In [22]:
skf = StratifiedKFold(n_splits = 10, shuffle = True, random_state=1)

In [23]:
confusion_matrixs = []
roc_curvs = []

In [25]:
for i, (train, test) in enumerate(skf.split(input_label, output_label)):
    print("Modelo " + str(i))
    inp_train, out_train = np.array(input_label)[train], np.array(output_label)[train]
    inp_test, out_test = np.array(input_label)[test], np.array(output_label)[test]
    model = createModel()
    model.fit(x = inp_train, y = out_train, validation_split= 0.1, epochs = 10, shuffle = True,verbose = 1)
    res = np.array([np.argmax(resu) for resu in model.predict(inp_test)])
    confusion_matrixs.append(confusion_matrix(out_test, res))
    fpr, tpr, _ = roc_curve(out_test,  res)
    auc = roc_auc_score(out_test, res)
    roc_curvs.append([fpr, tpr, auc])
    print("\n\n")

Modelo 0
Epoch 1/10
  742/26440 [..............................] - ETA: 9:18 - loss: 0.5811 - accuracy: 0.7312

KeyboardInterrupt: 

<h2>Roc Curves</h2>

In [ ]:
for i in range(10):
    print("------------------------------------")
    print("Modelo " + str(i))
    print(roc_curvs[i])
    print(confusion_matrixs[i])
    print("------------------------------------")